In [64]:
import hail as hl

In [2]:
import pandas as pd
import h5py
# import h5py_cache
import sys 
import numpy as np
sys.path.insert(0, '../code/')
from importlib import reload 
import my_hail_helper as myhelper

In [65]:
hl.init()

FatalError: Hail has already been initialized, restart session or stop Hail to change configuration.

In [3]:
myhelper = reload(myhelper)

In [4]:
covar, trait = myhelper.read_and_split_phenotype_csv(
    csv_path = '/vol/bmd/yanyul/GitHub/ptrs-ukb/output/query_phenotypes_cleaned_up.csv',  # '/Users/yanyul/Documents/repo/github/ptrs-ukb/output/query_phenotypes_cleaned_up.csv',
    pheno_names = 'ht,meaning',
    covar_names = 'pc1',
    indiv_id = 'eid',
    int_names = None,
    str_names = 'meaning,eid'
)

In [5]:
indiv_list = myhelper.read_indiv_list('/vol/bmd/yanyul/GitHub/ptrs-ukb/output/data_split/British-test-1.txt')

In [6]:
trait_sub = myhelper.subset_by_col(trait, 'eid', indiv_list)

In [7]:
indiv_in_sub_table = np.array(trait_sub['eid'].to_list()).astype('S10')

In [27]:
HDF5_CACHE = int(60 * (1024 ** 2))
f = h5py.File(
    '/vol/bmd/yanyul/UKB/predicted_expression/predicted_expression.ukb_imp_x_ctimp_Whole_Blood.h5', 
    'r'
)

In [28]:
f.keys()

<KeysViewHDF5 ['genes', 'pred_expr', 'samples']>

In [29]:
indiv_in_sub_table

array([b'1005003', b'1013480', b'1014217', ..., b'6017287', b'6022075',
       b'6023864'], dtype='|S10')

In [30]:
subset_idx_for_indiv_in_sub_table = np.where(np.isin(f['samples'][:], indiv_in_sub_table))

In [31]:
subset_idx_for_indiv_in_sub_table

(array([     1,      2,    129, ..., 487178, 487283, 487306]),)

In [32]:
subset_idx_for_indiv_in_sub_table[0].shape

(5000,)

In [33]:
# f['pred_expr'][1:100].shape
# np.array([i for i in range(10)])[::5]
list(subset_idx_for_indiv_in_sub_table[0])

[1,
 2,
 129,
 344,
 353,
 455,
 475,
 840,
 923,
 1003,
 1012,
 1099,
 1302,
 1394,
 1646,
 1841,
 2155,
 2257,
 2285,
 2677,
 2911,
 3035,
 3136,
 3161,
 3167,
 3226,
 3232,
 3250,
 3302,
 3312,
 3386,
 3395,
 3442,
 3707,
 3747,
 4018,
 4180,
 4192,
 4260,
 4262,
 4351,
 4397,
 4570,
 4579,
 4752,
 4825,
 4838,
 4867,
 4944,
 4986,
 5055,
 5534,
 5543,
 5784,
 5813,
 5827,
 5903,
 5973,
 6316,
 6378,
 6423,
 6669,
 6685,
 7013,
 7393,
 7484,
 7499,
 7545,
 7640,
 7815,
 7846,
 7992,
 8011,
 8071,
 8073,
 8160,
 8208,
 8239,
 8309,
 8454,
 8585,
 8596,
 8686,
 8730,
 8738,
 8791,
 8909,
 8957,
 8958,
 9300,
 9376,
 9378,
 9825,
 9910,
 9933,
 10063,
 10286,
 10313,
 10495,
 10546,
 10606,
 10680,
 10734,
 10754,
 10795,
 11169,
 11265,
 11455,
 11503,
 11519,
 11524,
 11705,
 11932,
 12002,
 12021,
 12149,
 12237,
 12359,
 12382,
 12541,
 12597,
 12681,
 12948,
 12962,
 13002,
 13150,
 13172,
 13206,
 13247,
 13420,
 13486,
 13513,
 13659,
 13674,
 13823,
 14071,
 14114,
 14179,
 142

In [34]:
pred_mat_sub = f['pred_expr'][:, list(subset_idx_for_indiv_in_sub_table[0])]  #

In [35]:
samples_sub = f['samples'][list(subset_idx_for_indiv_in_sub_table[0])]

In [37]:
pred_mat_sub[pred_mat_sub.sum(axis = 1) != 0,:].shape

(7041, 5000)

In [46]:
genes_sub = f['genes'][pred_mat_sub.sum(axis = 1) != 0]

In [47]:
pred_mat_sub.shape

(7044, 5000)

In [48]:
df = pd.DataFrame(pred_mat_sub[pred_mat_sub.sum(axis = 1) != 0,:])

In [49]:
df.columns = samples_sub.astype('str')
df.index = genes_sub.astype('str')

In [68]:
df['gene'] = df.index
df = df.reset_index(drop = True)

In [73]:
df.iloc[1:10,4990:5001]

,1937089,4882011,3683366,5577564,1984815,2637074,3686944,2969425,2874543,5129226,gene
1,-0.300679,-0.042579,0.005321,-0.272079,-0.143379,0.007621,-0.016579,-0.162279,-0.016179,-0.019779,ENSG00000000460.16
2,-0.023979,0.001421,0.002921,0.003821,0.001221,0.013821,-0.018179,-0.009979,-0.002479,-0.000679,ENSG00000001036.13
3,-0.002179,-0.006579,-0.014579,-0.017279,-0.052379,-0.083979,-0.143279,0.011321,-0.139179,-0.055679,ENSG00000001084.10
4,-0.087279,0.003021,0.139021,-0.158979,-0.208579,-0.264979,-0.147279,-0.052679,-0.160279,-0.153179,ENSG00000001460.17
5,-0.005479,0.027321,-0.029279,0.008721,0.055221,0.005521,0.017121,0.026521,0.048421,0.026321,ENSG00000001461.16
6,0.267321,0.053821,0.141521,-0.045579,0.078621,0.127921,0.216221,0.235021,0.150121,-0.058879,ENSG00000001561.6
7,0.027921,-0.223379,-0.257879,-0.009779,0.075221,-0.002579,-0.261679,-0.252579,0.040021,0.016721,ENSG00000002016.17
8,-0.227979,-0.016479,-0.467379,-0.225379,-0.467479,-0.224579,-0.004679,-0.457079,-0.461479,-0.222779,ENSG00000002549.12
9,-1.132979,-0.541679,-0.076979,-0.606279,-1.113679,-0.610079,-0.000279,-0.557779,-1.150679,-0.576879,ENSG00000002726.20


In [74]:
df.to_csv('test_output/pred_expr_british_test.tsv', index = None, header = True, sep = '\t')

In [60]:
df.iloc[1:10, 1:10]

,5172041,1528789,1517276,2468738,4556757,1484605,4185318,2261357,1221294
ENSG00000000460.16,-0.151485,0.041915,-0.108185,-0.053485,-0.050885,0.012115,0.044515,-0.155485,-0.045785
ENSG00000001036.13,0.009315,0.003315,-0.009485,0.010515,0.006815,0.012415,0.001615,-0.000285,0.001615
ENSG00000001084.10,-0.140685,-0.120785,-0.088485,-0.025185,-0.065785,-0.062785,-0.002185,-0.133985,-0.002085
ENSG00000001460.17,-0.073085,-0.068785,-0.229385,-0.114585,-0.077585,-0.157085,0.008715,0.006715,-0.072285
ENSG00000001461.16,0.002415,0.035015,0.049915,0.098715,0.050915,0.010215,-0.004685,0.029615,-0.006485
ENSG00000001561.6,0.125715,0.116915,0.002415,0.150215,0.113315,0.240915,-0.068085,0.059915,0.057715
ENSG00000002016.17,-0.006385,0.040915,-0.228485,0.020415,-0.232785,0.020615,-0.232585,-0.013585,-0.275485
ENSG00000002549.12,0.012715,-0.225085,-0.227785,-0.225685,-0.465885,-0.229185,-0.463785,0.010115,-0.466285
ENSG00000002726.20,-0.628785,-0.668985,-1.081285,-1.117985,-1.126385,-1.120085,-0.574485,-1.159485,-1.097385


In [75]:
test = hl.import_matrix_table('test_output/pred_expr_british_test.tsv')

FatalError: IOException: Stream is closed!

Java stack trace:
org.json4s.package$MappingException: unknown error
	at org.json4s.Extraction$.extract(Extraction.scala:43)
	at org.json4s.ExtractableJsonAstNode.extract(ExtractableJsonAstNode.scala:21)
	at org.json4s.jackson.Serialization$.read(Serialization.scala:50)
	at org.json4s.Serialization$class.read(Serialization.scala:30)
	at org.json4s.jackson.Serialization$.read(Serialization.scala:17)
	at is.hail.expr.ir.IRParser$.deserialize(Parser.scala:160)
	at is.hail.expr.ir.IRParser$.matrix_ir_1(Parser.scala:1277)
	at is.hail.expr.ir.IRParser$.matrix_ir(Parser.scala:1213)
	at is.hail.expr.ir.IRParser$$anonfun$parse_matrix_ir$2.apply(Parser.scala:1465)
	at is.hail.expr.ir.IRParser$$anonfun$parse_matrix_ir$2.apply(Parser.scala:1465)
	at is.hail.expr.ir.IRParser$.parse(Parser.scala:1449)
	at is.hail.expr.ir.IRParser$.parse_matrix_ir(Parser.scala:1465)
	at is.hail.expr.ir.IRParser$.parse_matrix_ir(Parser.scala:1464)
	at is.hail.expr.ir.IRParse

In [56]:
test = hl.Table.from_pandas(df.iloc[1:10, 1:10])

In [ ]:
mt_expr = test.to_matrix_table(row_key = ['eid'], col_key = ['gene'])

In [ ]:
mt_expr.describe()

In [57]:
test.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    '5172041': float64 
    '1528789': float64 
    '1517276': float64 
    '2468738': float64 
    '4556757': float64 
    '1484605': float64 
    '4185318': float64 
    '2261357': float64 
    '1221294': float64 
----------------------------------------
Key: []
----------------------------------------
